#### The problem: I have a set of thousands of polygons, which I need to summarize the percent of coverage from a very large (2GB)  raster. Zonal statistics tools can't be used; if a raster cell slightly overlaps one of the polygons, the entire pixel is counted in the statistics and the data are categorical.

#### The method is to create an individual raster for the each polygon (which end up being 1kb size) by getting the area, storing the data for each value as an array, and then creating a second array for the same clipped raster to join the values to the resulting area for that pixel in a pandas dataframe.

In [ ]:
import pycrs
from pycrs import parser
import fiona
from fiona.crs import from_epsg
from rasterio.crs import CRS
from rasterio.mask import mask
import rasterio
from shapely.geometry import box
import geopandas as gpd
import os
import numpy as np
import pandas as pd

In [ ]:
# the files to be processed
# ensure the tif & vector data are projected, to correctly calculate area.
data_dir = r'D:\data'    
rasterfile = os.path.join(r"D:\Data\mytif.tif")
pdata = gpd.GeoDataFrame.from_file(os.path.join(data_dir,'polygons.gpkg'))

#directory for output tiffs
ddir = r'D:\data\output'


In [ ]:
def getFeatures(gdf):
    # Function to parse features from GeoDataFrame in such a manner that rasterio wants them
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
# Create a new, smaller tif for each polygon in the dataset to speed up processing
# get the bounding box for each polygon in the geodataframe
pdata['minx'], pdata['miny'], pdata['maxx'], pdata['maxy'] = pdata.bounds['minx'], pdata.bounds['miny'],pdata.bounds['maxx'],pdata.bounds['maxy']

n = 0

with rasterio.open(rasterfile) as src:
    for feat in pdata.iterrows():
        # subset the gdf to process one polygon at a time
        gdf = pdata.iloc[n]
        n = n + 1
        out_name = gdf['id']
        
        # create bounding box to clip raster
        minx, miny, maxx, maxy = gdf['minx'], gdf['miny'], gdf['maxx'], gdf['maxy']
        bbox = box(minx, miny, maxx, maxy)
        geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(32610))
        coords = getFeatures(geo)
        
        # clip the raster based on the extent of the polygon
        print n, coords
        out_img, out_transform = mask(src, coords, crop=True, filled=False)
        
        # update metadata
        out_meta = src.meta.copy()
        crs = fiona.crs.from_epsg(32610)
        out_meta.update({"driver": "GTiff", "height": out_img.shape[1],"width": out_img.shape[2],"transform": out_transform,"crs": crs})
        
        # output the tiff
        out_tif = os.path.join(data_dir, 'tiffs',str(out_name)+'.tiff')
        with rasterio.open(out_tif, "w", **out_meta) as dest:
            dest.write(out_img)
        

In [ ]:
# Solution implemented from 
# https://gis.stackexchange.com/questions/281310/creating-a-raster-of-percent-of-pixel-occupied-from-shapefile/281343#281343
import ogr
def fractional_pixel_weights(fsrc, geom):

    gt = fsrc.transform
    xs = np.arange(gt[2], gt[2] +  gt[0]* (1 + fsrc.shape[1]), gt[0])
    ys = np.arange(gt[5], gt[5] +  gt[4]* (1 + fsrc.shape[0]), gt[4])

    # Convert geom into ogr geometry
    geom_ogr = ogr.CreateGeometryFromWkt(geom.to_wkt())

    # Loop through each grid cell, compute the intersecting area
    overlapping_areas = np.empty((len(ys)-1, len(xs)-1))
    for ix in range(len(xs)-1):
        xmin = xs[ix]
        xmax = xs[ix + 1]
        for iy in range(len(ys)-1):
            ymax = ys[iy]
            ymin = ys[iy + 1]

            # Intersecting area
            coords_wkt = "POLYGON ((" + str(xmin) + ' ' + str(ymax) + ', ' + str(xmax) + ' ' + str(ymax) + ', ' + str(xmax) + ' ' + str(ymin) + ', ' + str(xmin) + ' ' + str(ymin) + ', ' + str(xmin) + ' ' + str(ymax) + "))"
            polycell = ogr.CreateGeometryFromWkt(coords_wkt)

            overlapping_areas[iy, ix] = polycell.Intersection(geom_ogr).Area()
            

    # Ratio of overlapped area to pixel area
    #frac_intersected = (overlapping_areas / (abs(gt[0] * gt[4])))
    
    # Area of overlap polygon/pixels - 30m resolution
    frac_intersected = (overlapping_areas / (abs(gt[0] * gt[4]))) * 900

    return frac_intersected

In [ ]:
from osgeo import gdal
import sys

n = 0
area_totals = pd.DataFrame()
df = pd.DataFrame()

for feat in pdata.iterrows():
    # subset the gdf to process one polygon at a time
    gdf = pdata.iloc[n]
    fileid = gdf['id']
    
    # Open the clipped raster
    raster = rasterio.open(os.path.join(ddir, fileid +'.tiff'))
    
    # Set the geometry field to the polygon to be analyzed
    geofield = gdf['geometry']
    
    # Get pixel % coverage
    x = fractional_pixel_weights(raster, geofield)
    
    # Open the raster again to get the corresponding values in an array format
    raster = gdal.Open(os.path.join(ddir, fileid +'.tiff'))
    rarray = np.array(raster.GetRasterBand(1).ReadAsArray())
    
    np.set_printoptions(precision=2)
    print fileid#, x, rarray
    
    n = n + 1
    
    # Now iterate through each cell in both arrays and summarize the area for each category value per pixel
    nums = len(x)
    k = 0
    
    for ynum in range(0,nums):
        for xnum in range(0,nums):
            try:
                category = rarray[xnum,ynum]
                print str(xnum) +','+str(ynum)
                categoryarea = x[xnum,ynum]
                #print category, categoryarea
                
                df['Cateogry'] = category
                df['Area'] = categoryarea
                df['recID'] = fileid
                k = k + 1
                
                areatotals = df.append(pd.Series([k,category,categoryarea,fileid], index=df.columns), ignore_index=True)
                
                print str(k) + ' iterations'
            except IndexError as e:
                print 'next'
            area_totals = pd.concat([areatotals,area_totals])
            print area_totals.head()

In [ ]:
updated_totals = area_totals.groupby(['recID','Category'], as_index=False)[['Area']].sum()

In [ ]:
# pivot the values to get a table with the area for each category value by polygon id
update_totals = updated_totals.pivot(index=updated_totals.recID, columns='Category')['Area'].fillna(0)
print update_totals.head(20)
update_totals.to_csv(os.path.joins(data_dir,'pixel_summary.csv', index=False, sep=',')